In [1]:
import os
import shutil
if "tesi_magistrale" in os.listdir(os.curdir):
    shutil.rmtree("tesi_magistrale")
!git clone https://github.com/salvalai/tesi_magistrale.git

#!pip install keras-tuner
#!pip install prophet
#!pip install pmdarima

import sys
if "./tesi_magistrale" not in sys.path:
    sys.path.append("./tesi_magistrale")

Cloning into 'tesi_magistrale'...
remote: Enumerating objects: 1012, done.
remote: Counting objects: 100% (1012/1012), done.
remote: Compressing objects: 100% (645/645), done.
remote: Total 1012 (delta 598), reused 721 (delta 315), pack-reused 0
Receiving objects: 100% (1012/1012), 36.67 MiB | 11.71 MiB/s, done.
Resolving deltas: 100% (598/598), done.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

import os
tsdir = "/content/drive/MyDrive/magistrale/tesi/timeseries"
print(f"@{tsdir}: {os.listdir(tsdir)}")
groups = [(g, os.path.join(tsdir, g)) 
            for g in os.listdir(tsdir) 
            if os.path.isdir(os.path.join(tsdir, g))
        ]
groups

Mounted at /content/drive
@/content/drive/MyDrive/magistrale/tesi/timeseries: ['timeseries.zip', 'conversionTime.csv', 's1', 's2', 's3', 'repr_tsIDs.txt', 's11', 's10', 's9', 's8', 's4', 's5', 's6', 's7', 'hyperparams', 'extraction_selection.ipynb', 'training.ipynb', 'visualization.ipynb']


[('s1', '/content/drive/MyDrive/magistrale/tesi/timeseries/s1'),
 ('s2', '/content/drive/MyDrive/magistrale/tesi/timeseries/s2'),
 ('s3', '/content/drive/MyDrive/magistrale/tesi/timeseries/s3'),
 ('s11', '/content/drive/MyDrive/magistrale/tesi/timeseries/s11'),
 ('s10', '/content/drive/MyDrive/magistrale/tesi/timeseries/s10'),
 ('s9', '/content/drive/MyDrive/magistrale/tesi/timeseries/s9'),
 ('s8', '/content/drive/MyDrive/magistrale/tesi/timeseries/s8'),
 ('s4', '/content/drive/MyDrive/magistrale/tesi/timeseries/s4'),
 ('s5', '/content/drive/MyDrive/magistrale/tesi/timeseries/s5'),
 ('s6', '/content/drive/MyDrive/magistrale/tesi/timeseries/s6'),
 ('s7', '/content/drive/MyDrive/magistrale/tesi/timeseries/s7'),
 ('hyperparams',
  '/content/drive/MyDrive/magistrale/tesi/timeseries/hyperparams')]

In [3]:
import re
serie_csv_pattern = ".+.csv"
series = [(s[:-4], g_abs) 
            for g, g_abs in groups
            for s in os.listdir(g_abs) 
            if re.match(serie_csv_pattern, s) is not None
        ]

#import json
#print(json.dumps(series, indent = 4))
series

[('somma_progressiva', '/content/drive/MyDrive/magistrale/tesi/timeseries/s1'),
 ('SC_da_Media_Settimanale_Venduto',
  '/content/drive/MyDrive/magistrale/tesi/timeseries/s2'),
 ('conteggio_articoli',
  '/content/drive/MyDrive/magistrale/tesi/timeseries/s3'),
 ('venduto', '/content/drive/MyDrive/magistrale/tesi/timeseries/s3'),
 ('net_amount', '/content/drive/MyDrive/magistrale/tesi/timeseries/s11'),
 ('not_specified', '/content/drive/MyDrive/magistrale/tesi/timeseries/s10'),
 ('Classifica_su_venduto',
  '/content/drive/MyDrive/magistrale/tesi/timeseries/s9'),
 ('avg_selling_price', '/content/drive/MyDrive/magistrale/tesi/timeseries/s8'),
 ('Per_Venduto_Neg_Su_Tot',
  '/content/drive/MyDrive/magistrale/tesi/timeseries/s4'),
 ('Classifica_su_venduto',
  '/content/drive/MyDrive/magistrale/tesi/timeseries/s5'),
 ('not_specified', '/content/drive/MyDrive/magistrale/tesi/timeseries/s5'),
 ('ven_medio_cliente', '/content/drive/MyDrive/magistrale/tesi/timeseries/s6'),
 ('incidenza_sconti_ven',

In [4]:
"""Constants"""
serie_csv = "{}.csv"
predict_folder = "{}_prediction"
cluster_info = "{}_best_cluster.json"

serie_fig = "{}_{}.png"
prediction_fig = "{}.png"

In [5]:
import json
import time

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

def setup_plot():
    sns.set_style(style="whitegrid")
    #sns.set_style(style="darkgrid")
    plt.figure(figsize = (26, 6))
    plt.rcParams.update({'font.size': 15})
    plt.rcParams["xtick.labelsize"] = 15

from ml_microservice.anomaly_detection.transformers import Preprocessor
from ml_microservice.anomaly_detection.residual_analysis.empirical_rule import EmpiricalRule
from ml_microservice.anomaly_detection.models.windowed_gaussian import WindowedGaussian

def print_results(dimID, dimPath):
    cluster_f = cluster_info.format(dimID)
    with open(os.path.join(dimPath, cluster_f), "r") as fin:
        cluster = json.load(fin)
    print(f"[{dimID}] best cluster: id: {cluster['id']}, size: {cluster['size']}")

    df = pd.read_csv(os.path.join(dimPath, serie_csv.format(dimID)))
    ts = pd.DataFrame()
    ts["timestamp"] = pd.to_datetime(df["Date"])
    ts["value"] = df[cluster["id"]]
    ts = Preprocessor(ts).nan_filled
    test_1st_IDX = int(0.7 * len(ts))
    
    # Plot serie
    print(f"[{dimID}] Plot {cluster['id']}")
    setup_plot()
    sns.lineplot(
        x = "timestamp",
        y = "value",
        color = "#000000cc",
        data = ts,
    ).set_title(f"{dimID} : {cluster['id']}")

    plt.xticks(
        rotation=45, 
        horizontalalignment='right',
        fontweight='light',
        #fontsize='x-large'  
    )
    plt.axvspan(
        ts["timestamp"][0], 
        ts["timestamp"][test_1st_IDX], 
        color = "#c6b99380"
    )
    plt.xlabel("timestamp")
    plt.ylabel("valore")
    plt.tight_layout()
    
    fig_path = os.path.join(dimPath, serie_fig.format(dimID, cluster["id"]))
    plt.savefig(fig_path)
    #plt.show()
    plt.close()

    pred_dir = os.path.join(dimPath, predict_folder.format(dimID))
    models = [(f[:-4], os.path.join(pred_dir, f), )
                for f in os.listdir(pred_dir)
                if re.match(serie_csv_pattern, f) is not None
            ]
    for mID, pred_path in models:
        print(f"[{dimID}] Plot prediction for {cluster['id']}: {mID}")
        prediction = pd.read_csv(pred_path)
        prediction["timestamp"] = pd.to_datetime(prediction["timestamp"])
        pretty_dim = dimID if dimID != 'not_specified' else 'dimensione'

        setup_plot()
        sns.lineplot(
            x = "timestamp",
            y = "value",
            color = "#000000cc",
            data = ts.tail(len(prediction)),
        ).set_title(f"Anomalie rilevate mediante {mID} in {pretty_dim} : {cluster['id']}")

        plt.xticks(
            rotation=45, 
            horizontalalignment='right',
            fontweight='light',
        )
        if "forecast" in prediction.columns:
            sns.lineplot(
                x = "timestamp",
                y = "forecast",
                #color = "#00fe01",
                color = "b",
                data = prediction,
            )
        if "outlier" in prediction.columns and \
            "outlier_score" not in prediction.columns \
        :
            for i, outlier in enumerate(prediction["outlier"]):
                if not outlier:
                    continue
                plt.plot(
                    prediction["timestamp"][i], 
                    ts["value"][test_1st_IDX + i],
                    color = "r",
                    alpha = 0.8,
                    marker = 'o',
                )
        elif "outlier_score" in prediction.columns:
            for i, outlier_score in enumerate(prediction["outlier_score"]):
                if outlier_score < .95:
                    continue
                plt.plot(
                    prediction["timestamp"][i], 
                    ts["value"][test_1st_IDX + i],
                    color = "r",
                    alpha = 0.8,
                    marker = 'o',
                )
        plt.xlabel("timestamp")
        plt.ylabel("valore")
        plt.tight_layout()
        plt.savefig(os.path.join(pred_dir, prediction_fig.format(mID)))
        #plt.show()
        plt.close()

        if "forecast" in prediction.columns:
            print(f"[{dimID}] Plot residuals for {cluster['id']}: {mID}")
            prediction["residuals"] = ts["value"].tail(len(prediction)).to_numpy() - prediction["forecast"].to_numpy()
            
            setup_plot()
            sns.lineplot(
                x = "timestamp",
                y = "residuals",
                color = "r",
                data = prediction,
            ).set_title(f"Residui della predizione mediante {mID} in {pretty_dim} : {cluster['id']}")
            plt.xticks(
                rotation=45, 
                horizontalalignment='right',
                fontweight='light',
            )
            plt.xlabel("timestamp")
            plt.ylabel("residui")
            plt.tight_layout()
            plt.savefig(os.path.join(pred_dir, prediction_fig.format(mID + "_residuals")))
            #plt.show()
            plt.close()

            # Residual Analysis
            tmp = pd.DataFrame()
            tmp["value"] = prediction["residuals"]

            # - Empirical Rule
            er = EmpiricalRule(robust = True).fit(tmp)
            er_results = er.predict(tmp)

            setup_plot()
            sns.lineplot(
                x = "timestamp",
                y = "value",
                color = "#000000cc",
                data = ts.tail(len(prediction)),
            ).set_title(f"Anomalie rilevate mediante {mID} in {pretty_dim} : {cluster['id']}")
            plt.xticks(
                rotation=45, 
                horizontalalignment='right',
                fontweight='light',
            )
            sns.lineplot(
                x = "timestamp",
                y = "forecast",
                #color = "#00fe01",
                color = "b",
                data = prediction,
            )
            for i, v in enumerate(er_results["outlier"]):
                if not v:
                    continue
                plt.plot(
                    prediction["timestamp"][i], 
                    ts["value"][test_1st_IDX + i],
                    color = "r",
                    alpha = 0.8,
                    marker = 'o',
                )
            plt.xlabel("timestamp")
            plt.ylabel("valore")
            plt.tight_layout()
            plt.savefig(os.path.join(pred_dir, prediction_fig.format("{}_emp_rule".format(mID))))
            #plt.show()
            plt.close()

            # - Windowed Gaussian
            wg = WindowedGaussian(
                w = 52,
                step = 26
            )
            wg_results = wg.fit_predict(tmp)

            setup_plot()
            sns.lineplot(
                x = "timestamp",
                y = "value",
                color = "#000000cc",
                data = ts.tail(len(prediction)),
            ).set_title(f"Anomalie rilevate mediante {mID} in {pretty_dim} : {cluster['id']}")
            plt.xticks(
                rotation=45, 
                horizontalalignment='right',
                fontweight='light',
            )
            sns.lineplot(
                x = "timestamp",
                y = "forecast",
                #color = "#00fe01",
                color = "b",
                data = prediction,
            )
            for i, outlier_score in enumerate(wg_results["outlier_score"]):
                if outlier_score < .95:
                    continue
                plt.plot(
                    prediction["timestamp"][i], 
                    ts["value"][test_1st_IDX + i],
                    color = "r",
                    alpha = 0.8,
                    marker = 'o',
                )
            plt.xlabel("timestamp")
            plt.ylabel("valore")
            plt.tight_layout()
            plt.savefig(os.path.join(pred_dir, prediction_fig.format("{}_win_gauss".format(mID))))
            #plt.show()
            plt.close()


#print_results(*series[0])

In [6]:
import json
import multiprocessing as mp
import os

try:

    with mp.Pool(processes = 10) as pool:
        r = pool.starmap(print_results, series)
    #print(json.dumps(r, indent = 4))
except Exception as e:
    print("Catched")
    pool.join()
    raise e

[conteggio_articoli] best cluster: id: 01047794, size: 3
[net_amount] best cluster: id: net_amount, size: 1
[Classifica_su_venduto] best cluster: id: 0025, size: 6
[venduto] best cluster: id: 01046318, size: 3
[SC_da_Media_Settimanale_Venduto] best cluster: id: 4, size: 2
[not_specified] best cluster: id: B, size: 2
[Per_Venduto_Neg_Su_Tot] best cluster: id: 0090, size: 4
[somma_progressiva] best cluster: id: 1, size: 2
[avg_selling_price] best cluster: id: 226.378, size: 41
[Classifica_su_venduto] best cluster: id: 226.571, size: 35
[Classifica_su_venduto] Plot 0025
[not_specified] Plot B
[net_amount] Plot net_amount
[SC_da_Media_Settimanale_Venduto] Plot 4
[somma_progressiva] Plot 1
[Per_Venduto_Neg_Su_Tot] Plot 0090
[Classifica_su_venduto] Plot prediction for 0025: deepant
[not_specified] Plot prediction for B: deepant
[SC_da_Media_Settimanale_Venduto] Plot prediction for 4: deepant
[net_amount] Plot prediction for net_amount: deepant
[somma_progressiva] Plot prediction for 1: deepa

In [8]:
import json
import re
import yaml

from tqdm import tqdm

cluster_info = "{}_best_cluster.json"
repr_ext = "{:s}_repr_tsIDs.txt"

def extraction_table(groups):
    table = []
    row = " {} & {} & {} & {} & {} \\\\"
    table.append(row.format(
        "Gruppo",
        "Serie selezionata",
        "Post \\acr{PCA}",
        "Cluster",
        "Tempo Totale (s)"
    ))
    table.extend(["\\hline"]*2)

    for gID, path in tqdm(groups):
        #print(f"[{gID}] Path: {path}")
        series = [(s[:-15], repr_ext.format(s[:-15]), cluster_info.format(s[:-15]), )
                    for s in os.listdir(path)
                    if re.match(repr_ext.format(".+"), s) is not None
                ]
        for s, repr_txt, cluster_json in series:
            #print(f"[{gID}][{s}] Cluster: {cluster_json}, Info: {repr_txt}")
            with open(os.path.join(path, repr_txt), "r") as fin:
                r = yaml.load(fin)
            with open(os.path.join(path, cluster_json), "r") as fin:
                c = json.load(fin)
            table.append(row.format(
                gID[1:],
                c["id"],
                r["PCA"]["Reduced dimensions"] if "PCA" in r.keys() else "/",
                c["size"],
                r["Time (s)"],
            ))
    print("\n" + "\n".join(table))

extraction_table(groups)

100%|██████████| 12/12 [00:27<00:00,  2.32s/it]


 Gruppo & Serie selezionata & Post \acr{PCA} & Cluster & Tempo Totale (s) \\
\hline
\hline
 1 & 1 & 2 & 2 & 0.05360293388366699 \\
 2 & 4 & 4 & 2 & 0.05198216438293457 \\
 3 & 01047794 & 3 & 3 & 2.382472038269043 \\
 3 & 01046318 & 1 & 3 & 1.9086782932281494 \\
 11 & net_amount & / & 1 & 0.003173351287841797 \\
 10 & B & 1 & 2 & 0.017436504364013672 \\
 9 & 226.571 & 53 & 35 & 8.134458780288696 \\
 8 & 226.378 & 1 & 41 & 3.167372226715088 \\
 4 & 0090 & 3 & 4 & 1.5622811317443848 \\
 5 & 0025 & 37 & 6 & 1.601362943649292 \\
 5 & 0062 & 2 & 4 & 1.2849738597869873 \\
 6 & 0096 & 1 & 3 & 0.38825511932373047 \\
 7 & 01050617 & 6 & 2 & 2.062976598739624 \\


In [9]:
import json
import re
import yaml

from tqdm import tqdm

cluster_info = "{}_best_cluster.json"
repr_ext = "{:s}_repr_tsIDs.txt"
hyper_info = "{}_hypertuning.json"
scores_tab = "scores_tab.txt"

def scores_table(dimID, dimPath):
    print(f"[{dimID}] Table for {dimID}")
    table = []
    row = " {} & {} & {} & {} & {} \\\\"
    table.append(row.format(
        "Metodo",
        "MSE",
        "RMSE",
        "Naive",
        "Tempo Totale (s)"
    ))
    table.extend(["\\hline"]*2)
    
    pred_dir = os.path.join(dimPath, predict_folder.format(dimID))
    methods_results = [(f[:-17], os.path.join(pred_dir, f),)
                       for f in os.listdir(pred_dir)
                       if re.match(hyper_info.format(".+"), f) is not None
                    ]
    
    for mID, hyper_file in methods_results:
        if mID == "wingauss":
            continue
        with open(hyper_file, "r") as fin:
            hyper_res = json.load(fin)
        table.append(row.format(
            mID,
            "{:.4f}".format(hyper_res["scores"]["mse"]),
            "{:.4f}".format(hyper_res["scores"]["rmse"]),
            "{:.4f}".format(hyper_res["scores"]["naive"]),
            "{:.4f}".format(hyper_res["tune_total_time"]),
        ))
    
    with open(os.path.join(pred_dir, scores_tab), "w") as fout:
        fout.write("\n".join(table))

#scores_table(*series[0])
import json
import multiprocessing as mp
import os

try:
    with mp.Pool(processes = 10) as pool:
        r = pool.starmap(scores_table, series)
    #print(json.dumps(r, indent = 4))
except Exception as e:
    print("Catched")
    pool.join()
    raise e

[SC_da_Media_Settimanale_Venduto] Table for SC_da_Media_Settimanale_Venduto
[somma_progressiva] Table for somma_progressiva
[conteggio_articoli] Table for conteggio_articoli
[net_amount] Table for net_amount
[venduto] Table for venduto
[avg_selling_price] Table for avg_selling_price
[Classifica_su_venduto] Table for Classifica_su_venduto
[Classifica_su_venduto] Table for Classifica_su_venduto
[not_specified] Table for not_specified
[Per_Venduto_Neg_Su_Tot] Table for Per_Venduto_Neg_Su_Tot
[not_specified] Table for not_specified
[ven_medio_cliente] Table for ven_medio_cliente
[incidenza_sconti_ven] Table for incidenza_sconti_ven


In [10]:
import json
import os
import re

serie_csv_pattern = ".+.csv"
hyperparams_folder = "hyperparams"
table_file = "{}.txt"

def hyperparams_tables(groups, wdir):
    h_dir = os.path.join(wdir, hyperparams_folder)
    if not os.path.exists(h_dir):
        os.makedirs(h_dir)
    head = {
        "deepant": ["Gruppo", "Strati", "Filtro", "Mappe", "Stride", "Dimensione", "Dropout Rate"],
        "gru": ["Gruppo", "Dimensione 1", "Dropout Rate", "Dropout Rate Stato", "Dimensione 1", "Dropout Rate", "Dropout Rate Stato"],
        "lstm": ["Gruppo", "Dimensione 1", "Dropout Rate", "Dropout Rate Stato", "Dimensione 1", "Dropout Rate", "Dropout Rate Stato"],
        "sarimax": ["Gruppo", "p", "d", "q", "P", "D", "Q"],
        "prophet": ["Gruppo", "Changepoint Prior Scale", "Stagionality Prior Scale"],
    }
    body = {
        "deepant": [],
        "gru": [],
        "lstm": [],
        "sarimax": [],
        "prophet": [],
    }

    for gID, gPath in groups:
        ps = [(p[:-11], os.path.join(gPath, p), )
            for p in os.listdir(gPath) 
            if re.match(predict_folder.format(".+"), p) is not None
        ]
        for pred, predPath in ps:
            for mID in body.keys():
                
                if hyper_info.format(mID) not in os.listdir(predPath):
                    print(f"{gID} - {pred} : MISSING {mID}")
                    continue
                print(f"{gID} - {pred} : {mID}")
                with open(os.path.join(predPath, hyper_info.format(mID)), "r") as fin:
                    hps = json.load(fin)
                hps = hps["best_config"]
                if mID == "deepant":
                    body["deepant"].append([
                                            "{}_{}".format(gID, pred),
                                            hps["conv_layers"],
                                            hps["kernel"],
                                            hps["maps"],
                                            hps["conv_strides"],
                                            hps["hidden_size"],
                                            hps["dropout_rate"]
                    ])
                elif mID == "sarimax":
                    body["sarimax"].append([
                                            "{}_{}".format(gID, pred),
                                            hps["order"][0],
                                            hps["order"][1],
                                            hps["order"][2],
                                            hps["seasonal_order"][0],
                                            hps["seasonal_order"][1],
                                            hps["seasonal_order"][2],
                    ])
                elif mID == "prophet":
                    body["prophet"].append([
                                            "{}_{}".format(gID, pred),
                                            hps["changepoint_prior_scale"],
                                            hps["seasonality_prior_scale"],
                    ])
                elif mID == "gru":
                    body["gru"].append([
                                        "{}_{}".format(gID, pred),
                                        hps["size1"],
                                        hps["dropout1"],
                                        hps["rec_dropout1"],
                                        hps["size2"],
                                        hps["dropout2"],
                                        hps["rec_dropout2"]
                    ])
                elif mID == "lstm":
                    body["lstm"].append([
                                        "{}_{}".format(gID, pred),
                                        hps["size1"],
                                        hps["dropout1"],
                                        hps["rec_dropout1"],
                                        hps["size2"],
                                        hps["dropout2"],
                                        hps["rec_dropout2"]
                    ])
    for mID in head.keys():
        print(f"Composing {mID} table")
        table = []
        rowF = "&".join([" {} "]*len(head[mID]))
        rowF += "\\\\"
        
        table.append("\\begin{tabular}{" + "|".join(["c"]*len(head[mID])) + "}")
        table.append(rowF.format(*head[mID]))
        table.extend(["\\hline"]*2)
        for line in body[mID]:
            table.append(rowF.format(*line))
        table.append("\\hline")
        table.append("\\end{tabular}")

        table_path = os.path.join(h_dir, table_file.format(mID))
        with open(table_path, "w") as fout:
            fout.write("\n".join(table))

hyperparams_tables(groups, tsdir)

s1 - somma_progressiva : deepant
s1 - somma_progressiva : gru
s1 - somma_progressiva : lstm
s1 - somma_progressiva : sarimax
s1 - somma_progressiva : prophet
s2 - SC_da_Media_Settimanale_Venduto : deepant
s2 - SC_da_Media_Settimanale_Venduto : gru
s2 - SC_da_Media_Settimanale_Venduto : lstm
s2 - SC_da_Media_Settimanale_Venduto : sarimax
s2 - SC_da_Media_Settimanale_Venduto : prophet
s3 - conteggio_articoli : deepant
s3 - conteggio_articoli : gru
s3 - conteggio_articoli : lstm
s3 - conteggio_articoli : sarimax
s3 - conteggio_articoli : prophet
s3 - venduto : deepant
s3 - venduto : gru
s3 - venduto : lstm
s3 - venduto : MISSING sarimax
s3 - venduto : prophet
s11 - net_amount : deepant
s11 - net_amount : gru
s11 - net_amount : lstm
s11 - net_amount : sarimax
s11 - net_amount : prophet
s10 - not_specified : deepant
s10 - not_specified : gru
s10 - not_specified : lstm
s10 - not_specified : sarimax
s10 - not_specified : prophet
s9 - Classifica_su_venduto : deepant
s9 - Classifica_su_venduto 